# Test notebook to PDF
here is some example text

In [5]:
from pathlib import Path

import pandas as pd
import arcpy

fc_ctypes_in = r'I:\Projects\Darren\PPA3_GIS\PPA3_GIS.gdb\Subcommtypes_2025blueprint_final'
f_ctype_start = 'comtyp_ppa_in'
f_jur = 'Jurisdiction'

config_csv = r'commtyp_ppa_config.csv'
f_cfg_jur = 'jur'
f_cfg_ctyp_plan = 'ctyp_plan'
f_cfg_ctypfinal = 'ctyp_ppa'

df_cfg = pd.read_csv(config_csv)

f_ctyp_final = 'comtyp_ppa_fin'
fc_fields = [f.name for f in arcpy.ListFields(fc_ctypes_in)]
if f_ctyp_final not in fc_fields:
    arcpy.management.AddField(fc_ctypes_in, f_ctyp_final, field_type='TEXT')
    
fields = [f_ctype_start, f_jur, f_ctyp_final]
with arcpy.da.UpdateCursor(fc_ctypes_in, field_names=fields) as ucur:
    for row in ucur:
        ctplan = row[fields.index(f_ctype_start)]
        jur = row[fields.index(f_jur)]
        
        lkp_results = df_cfg.loc[(df_cfg[f_cfg_ctyp_plan] == ctplan) \
                           & (df_cfg[f_cfg_jur] == jur)][f_cfg_ctypfinal].values
        
        ctppa = ctplan # by default, final ctype is plan ctype
        if len(lkp_results) > 0: # but if specified in config CSV, use that value for final ctype
            ctppa = lkp_results[0]
        row[fields.index(f_ctyp_final)] = ctppa
        ucur.updateRow(row)
 
print("update completed.")

update completed.
